In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
import openai
import os

In [ ]:
os.environ["OPENAI_API_KEY"]= "Your OpenAI API Key"

In [ ]:
directory = ["""Genpact Privileged access Management (PAN) PRODUCTION Windows end user manual
 
BeyondTrust Privilege Access Management 
Genpact - Windows end user Manual
 
This is a draft for discussion only. It represents work in progress and may contain preliminary information which is subject to change. 
This draft is subject to our Quality Assurance Review.
 
Document control
0

1.Introduction
 
1.1 Purpose 
This document describes the windows end user login process to PAM Solution using domain credentials/OHR ID Okta which will provide access to Powerbroker Password safe portal. This also covers procedures of accessing the target servers
 
1.2 Scope
This document includes the ways to access the target systems using various managed accounts, requesting elevation to more privileged managed accounts, approving or rejecting the requested session and various other useful features of the tool for the users.

1.3 Audience
This document is intended for PAM Administrators who will log on to PAM solution and troubleshoot the PAM managed various target systems.

2. Login procedure for auto Approved Requests
 
2.1 Enter the Okta URL (https://genpact.okta-emea.com/) in the browser (Edge/Chrome is recommended)
 
2.2 Log in with OHR ID & Password in okta

2.3 After successful login to okta, Add PAM-AWS app from Genpact Apps and click on the tile as shown helow.
 
2.4 Upon clicking on the PAM-AWS, user will be redirected to enter Passcode or send Push notification.
 
2.5 Once the passcode or Push notification authentication is done user will be Redirected to PAM (Beynndtrust) web portal.
 
2.6 After successful logging in to BT, favorites tab will be displayed by default and user would be able to access other required tabs.
 
1. Click on Domain Linked Accounts as devices like Windows, network which are accessed using domain credentials will be shown here
 
2. Click on Click here to return all accounts to view all the available servers & devices mapped with various managed accounts
 
3. Devices can be searched using search field given in top-right area, * will indicate all devices.
 
4. Search can be further refined using funnel shaped icon given in the top row
Note: user can select the L1, L2, L3 account as per the role. Any user looking for a higher role would need approval and request will be in pending state till it is approved ar rejected.

5. There are two ways to access open the sessions; one is to open normally while another is by using one click feature for quick access.
Note: One-Click feature is only available for Auto Approved requests. The icon (flash) would be would black while in case of approval it would be greyed out & disabled.
 
6. Approach One Using normal way: 
a. Click on the Access icon (in right) aside of the account and device details         which you want to access using the required account mapped against it

b. This will Inad the page to request submission form
 
c. Click on Submit Request after filling it as per need and mentioning the valid reason for the request 
Note: Reason must be valid else request will be rejected in case of invalid reason and will be scrutinize during the auditing.
 
d. This will be redirected to Requests page and request can be seen in the approved status tab if approved or auto-approved. The request will be available here for the time duration it has been requested for
 
e. Click on the active request (3 vertical dots) to Start Session.
 
f. Click on start Session to open the session & it will download the RDP session
 
7. Approach Two using Quick Launch 
a. Click on the Quick Launch icon in the row same as that of required device & managed account which user wants to access

b. Mention the Requested Duration & Reason in Flash screen pop up
 
c. Click on Start SSH session to open the session & it will sunload the RDP session

d. Open the saved session & a new screen for RDP will open up and user can perform tasks as per role defined and permissions enabled for the managed account used to open the RDP Session
 
e. Once tasks are performed, session can be logged off but the request in Password Safe will remain for the user for the requested duration
 
f. This will consume a session out of available session as mentioned in the Status tab
 
Red - No session available; Orange - Some sessions available, Green - All or unlimited Sessions available

g. To free up the consumed session, open the active request and Check-in Request
 

 
3. Login Procedure for Requests Requiring Approval
 
1. Follow steps from 1-16 as mentioned in 3.
 
2. Submit the request using approach 1 i.e. normal way (One-Click would be disabled for such requests) and you will be redirected to Requests page
 
3. Request would be seen in the Pending tab
 
4. Open the request to see the status and approver of the request
 
5. Approver can login using his/her credentials and go to Approve page & click on requests; in Pending tab
 
6. On the basis of authenticity/validity of the request, user can Approve or Deny/Cancel the request
 
7. If approved, the request's status will change for the user and request can be seen in Active tab
a. Click on the request to view the approver, comments & open RDP session
b. Open RDP Session & rest of the steps sould be same as steps 8-11 from 3.1 as mentioned above
 
8. If not approved, the request's status will change for the user and request can be seen in All tab
a. Click on the request to view the approver (rejector) and comments to reject the request.
 
9. All user requests Rejected/Approved/Checked-In can be seen in All tab.
 
4. FAQ
 
Question: Which are the different options/but given on Password safe Portal home page? 
Answer: Password Safe mortal's home page looks like:
1. Menu - Opens up the menu elaborately 
2. Home - Home Page
3. Password safe - Password Safe Page i.e. Home Page for End User
4. Favourites - Lists favourites selected devices & accounts combinations (Star marked)
5. Recently Used - Lists recently used devices
6. Systems - Lists systems and devices which are not in domain (For ex: Linux)
7. Databases Lists database devices (for ex: Oracle & sql)
8. Domain Linked Accounts - Lists devices which are in domain (for ex: Windows & Netwrk)
9. Applications - Lists devices which use other application like sql Plus, Putty (as an application) 
10. Cloud - Lists cloud devices
11. Accounts - Opens up the accounts available used to access devices
12. User Login - Shows user logged in and logout options
13. Settings - changes portal's page by changing color schirme
14. Requests - Redirects All, Active & Pending requests' tabs
15. System - Displays hostnames
16. Directory - Displays domains
17. Account - Managed accounts
18. Account Description - Description of managed accounts
19. Status - Number of sessions available (Green-ALL, Yellow-Partial, Red- None)
20. Platform - Displays flavour of the operating System or type of devices/system
21. Application - Applications used apart from RDP or SSH session
22. Workgroup - Workgroup of the system 
23. Filter - Filter out using any search patterns,
24. About - About the portal and contains official documents
There are no favourites selected but in the below some random favourites have been selected:
25. One Click Launch - one Click Launch feature is for quicker access but is available only in case the request is auto approved.


Question: Which are the different options/buttons given on Password Safe Portal request submission page? 
Answer: Password Safe Portal's request submission page looks like:
1. Account - Name of the managed account used to login on which managed system
2. Start Date - Date for the request's start 
3. Access Policy Window - Access policy used for the particular account
4. Start Time - Requested duration's start time. It can be immediate or scheduled for later time.
5. Requested Duration - Duration for which the request would be active. By default, it is 2 hours and can be increased but it should not expand to the next day i.e. it should not start on 7th Sep and end on 8th Sep
6. Access Request - Type of access requested
7. Reason - A valid reason why this access is needed. 
8. Ticket System - Ticket system in use in the environment
9. Ticket Number - a valid ticket number
10. Submit Request - Request submission button
 
Question: How is End user login page experience of a Requester different from an Approver? 
Answer: Below page is for a Requester which only contains Accounts page & Requests page
1. Accounts - Displays all the available managed accounts & managed devices. 2. Requests - Displays all the Active, Pending, Cancelled and Expired requests.
Below page is for an Approver (Requester Approver) which contains approve page as well alongwith Accounts & Requests
3. Approve - Displays requests which requestes are pending for approval
 
Question: What is bequested Duration?
Answer: Requested Daration is the duration the request would be active for a user in Requestes page. Ideally, user should not open another request during the same duration and should wait
 
Question: How many sessions, a user can open for the same request?
Answer: User can have maximum 2 sessions as depending upon RDP sessions limit on a particular machine well as the configuration and privilege but ideally there should be single session
 
Question: What will happen if the Requested duration is exhausted?
Answer: There are 2 scenarios:
1. In case, the duration exahuasted while the user is working on an open session, the session will continue till it is locked out. It is only the request which will get expired not the case. 
2. In case, the duration exahausted while there is no session opened by the user, the request will expire and user need to raise another request if he/she needs to open another session

How is and User Login page experience of a Requester different from an Approver? Below page is for a Requester which only contains Accounts page & Requests page
 
1. Accounts Displays all the available managed accounts & managed devices. 2. Requests Displays all the Artive, Pending, Cancelled and expired requests.
 
#elow page is for an Approver (Requester Approver) which contains Approve page as well along with accounts & requests
 
3. Approve Displays requests which requestes are penting for approval
 
Question: What is Requested Duration?
 
Answer: Requested Duration is the duration the request would be active for a imer in Requestes page. Ideally, oser should not spen another request during the same duration and should wait
 
Question: How many sessions a user can open for the same request?
 
Answer: User can have maximum 2 sessions as depending upon DP sessions limit on a particular machine as well as the configuration and privilege but ideally there should be single session
 
Question: What happens if the session becomes idle & locks out within Requested duration? 
Answer: If the session becomes idle & locks out within the Requested Duration, the user can always go Back to the Requests page open the session from the request in the Active tab.
 
Question: How can user extend the session once the request is expired? 
Answer: If the request is expired and user wants to extend the session, user needs to raise a new request.

"""]
def load_docs(directory):
    # loader = TextLoader(directory)
    # documents = loader.load()
    return directory

documents = load_docs(directory)
print(documents)

In [ ]:
def split_docs(documents, chunk_size=1000, chunk_overlap=50):
    text_split = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_split.create_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

In [ ]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)

In [ ]:
query = "What is the Login Procedure for Requests Requiring Approval?"
matching_docs = db.similarity_search(query)
matching_docs[0]

In [ ]:
matching_docs2 = db.similarity_search_with_score(query,k=3)
matching_docs2

In [ ]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_directory)
vectordb.persist()

In [ ]:
new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
matching_final = new_db.similarity_search_with_score(query,k=3)
matching_final

In [ ]:
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query_final = "What is Ticket System?"
matching_docs_final = new_db.similarity_search(query_final)
# Ensure 'question' key is included in the input dictionary
input_data = {'question': query_final, 'input_documents': matching_docs_final}
answer = chain.invoke(input=input_data)
answer

In [ ]:
print(answer['output_text'])